In [1]:
import os
import sys  
sys.path.insert(0, r'C:\Users\Rodrigo\Desktop\PhD\Study1\Physiological_Data')

In [85]:
import warnings
from Signals_Processing import *
from Process import *
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn import ensemble, feature_selection
import pickle

In [3]:
"""Load Data from Folder"""

folder = os.getcwd() + "\\Training Models\\"
participant = "P2_testing"
path = folder + participant

In [4]:
os.getcwd()

'C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system'

In [41]:
os.chdir(path)
(
    users,
    EEG_epochs,
    EEG_filtered,
    Signals_epochs,
    Signals,
    EEG_dict,
    features_signals,
    features_EEG,
    features_epochs,
    features_epochs_EEG,
) = ({}, {}, {}, {}, {}, {}, {}, {}, {}, {})

for root, dirs, files in os.walk(path):
    for fname in files:
        if fname.endswith(".xdf"):
            users[fname] = Run_files(fname)

In [42]:
users

{'P2_testing.xdf': [{'info': defaultdict(list,
               {'name': ['PsychoPy Markers'],
                'type': ['Markers'],
                'channel_count': ['2'],
                'channel_format': ['string'],
                'source_id': ['Example'],
                'nominal_srate': ['0.000000000000000'],
                'version': ['1.100000000000000'],
                'created_at': ['253468.5219258000'],
                'uid': ['6c3fbb93-2bc9-4cfe-9208-134490b2bf7b'],
                'session_id': ['default'],
                'hostname': ['DESKTOP-8HVM7BE'],
                'v4address': [None],
                'v4data_port': ['16573'],
                'v4service_port': ['16573'],
                'v6address': [None],
                'v6data_port': ['16573'],
                'v6service_port': ['16573'],
                'desc': [None],
                'stream_id': 3,
                'effective_srate': 0}),
   'footer': {'info': defaultdict(list,
                {'first_timestamp'

In [43]:
Opensignals_fs = 100
EEG_fs = 250
resolution = 16

In [44]:
sensors = ["ECG", "EDA", "RESP"]
data = {}
for user in users.keys():
    data[user.split(".")[0]] = Load_Data(
        users[user],
        "OpenSignals",
        "openvibeSignal",
        "PsychoPy Markers",
        "PsychoPy Markers",
        sensors,
    )

    data[user.split(".")[0]]["Markers"].pop(0)
    data[user.split(".")[0]]["Markers"].pop(-1)
    data[user.split(".")[0]]["Markers Timestamps"].pop(0)
    data[user.split(".")[0]]["Markers Timestamps"].pop(-1)

onset, offset, videos, valence, arousal = ({}, {}, {}, {}, {})
for user in data.keys():
    valence[user] = data[user]["Valence"]
    arousal[user] = data[user]["Arousal"]
    onset[user], offset[user], videos[user] = getMarkers(
        data[user]["Markers"], data[user]["Markers Timestamps"]
    )

onset_index = {}
offset_index = {}
onset_index_EEG = {}
offset_index_EEG = {}

for user in data.keys():
    onset_index[user], offset_index[user] = getMarkersIndex(
        onset[user], offset[user], data[user]["Signals"]["Time"]
    )
    onset_index_EEG[user], offset_index_EEG[user] = getMarkersIndex(
        onset[user], offset[user], np.array(data[user]["EEG"]["Time"])
    )

events_diff = {}

for keys in onset.keys():
    events_diff[keys] = CalculateEventsDiff(onset[keys], offset[keys])

In [45]:
"""EEG Processing"""

EEG_filtered = filterEEG(data, EEG_fs)
EEG_filtered = getEEGChannels(EEG_filtered)
EEG_epochs = getEpochs(EEG_filtered, onset_index_EEG, events_diff, EEG_fs)
EEG_dict = getVideosDict(EEG_epochs, videos)
features_EEG = getEEGBands(EEG_dict, EEG_fs)

In [46]:
features_EEG

{'P2_testing': {'EMDB/Horror/1000.avi': {'EEG_1': {'alpha': 8.199765295993416e-05,
    'betha': 6.45203945349297e-05,
    'gamma': 4.555476443663055e-05,
    'theta': 0.00029896967933332836},
   'EEG_2': {'alpha': 0.00015496242591648152,
    'betha': 0.00015013345040289385,
    'gamma': 9.790160138780468e-05,
    'theta': 0.000360690915548123},
   'EEG_3': {'alpha': 0.00010438022091281541,
    'betha': 0.0001817518670103483,
    'gamma': 0.00019589939107991788,
    'theta': 0.00022614642555801725},
   'EEG_4': {'alpha': 1.0382086267985248e-05,
    'betha': 1.1401343358740367e-05,
    'gamma': 8.27731610979492e-06,
    'theta': 2.0057801760648677e-05},
   'EEG_5': {'alpha': 0.0002179149135729684,
    'betha': 0.00022401002769719137,
    'gamma': 0.00015079651233886052,
    'theta': 0.00039945192254868295},
   'EEG_6': {'alpha': 1.5557510616279318e-05,
    'betha': 2.5025219535926193e-05,
    'gamma': 2.392610030397917e-05,
    'theta': 2.8830488152292687e-05},
   'EEG_7': {'alpha': 1.75

In [47]:
EEG_baseline = pd.read_csv(r'C:\Users\Rodrigo\Desktop\PhD\loop-system\Training Models\P2\EEGbaseline.csv', sep=";", index_col=0)

In [48]:
EEG_baseline

,EEG_1_alpha,EEG_1_betha,EEG_1_gamma,EEG_1_theta,EEG_2_alpha,EEG_2_betha,EEG_2_gamma,EEG_2_theta,EEG_3_alpha,EEG_3_betha,...,EEG_30_gamma,EEG_30_theta,EEG_31_alpha,EEG_31_betha,EEG_31_gamma,EEG_31_theta,EEG_32_alpha,EEG_32_betha,EEG_32_gamma,EEG_32_theta
0,0.000007,0.000006,0.000005,0.000013,0.000002,0.000003,0.000002,0.000003,0.000028,0.00004,...,0.000049,0.000075,4.191951e-07,6.191523e-07,5.605480e-07,4.045605e-07,0.000003,0.000003,0.000003,0.000002


In [49]:
EEG_list = EEG_baseline.loc[0].values.flatten().tolist()
print(EEG_list)

[7.309503971944594e-06, 6.25382996776262e-06, 5.194849961451926e-06, 1.299925765007474e-05, 2.1673254077541485e-06, 3.035028910171359e-06, 2.397354319538596e-06, 2.504801547287941e-06, 2.7879776602357826e-05, 4.000373450921519e-05, 7.510997486095257e-05, 3.678794144674948e-05, 9.237599992025294e-07, 1.7062283172507529e-06, 1.2641134010116468e-06, 8.666368342734513e-07, 6.2947743743116165e-06, 7.819412145117043e-06, 6.480355814570442e-06, 2.9591964900892342e-06, 1.3748246854567937e-06, 1.610727815436535e-06, 2.55850521150278e-06, 1.6702763175735729e-06, 1.7136133868165737e-06, 3.417692007060671e-06, 2.987547503812162e-06, 1.8011792778468414e-06, 3.1941970588771924e-06, 3.5093460451977336e-06, 2.9365733111381056e-06, 1.3908497830858512e-06, 6.879877845240648e-07, 5.486432495740666e-07, 4.503647242169412e-07, 3.858608567710675e-07, 2.03193640077416e-06, 2.3949304457753324e-06, 2.4239781187337186e-06, 1.1458575267922458e-06, 2.517411942875609e-06, 2.3201672930673744e-06, 1.98621806501872e-

In [50]:
Signals_epochs = getSignalsEpochs(data, onset_index, events_diff, Opensignals_fs)
Signals = getVideosDict(Signals_epochs, videos)
features_signals = getFeatures(Signals, Opensignals_fs, resolution)

C:\ProgramData\Anaconda3\envs\Loop System\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 128 is greater than input length  = 5, using nperseg = 5
  warnings.warn('nperseg = {0:d} is greater than input length '
C:\Users\Rodrigo\Desktop\PhD\Study1\Physiological_Data\lib\sensors.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ratio = round(lf_norm / hf_norm, 2)
C:\ProgramData\Anaconda3\envs\Loop System\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 128 is greater than input length  = 5, using nperseg = 5
  warnings.warn('nperseg = {0:d} is greater than input length '
C:\Users\Rodrigo\Desktop\PhD\Study1\Physiological_Data\lib\sensors.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ratio = round(lf_norm / hf_norm, 2)
C:\ProgramData\Anaconda3\envs\Loop System\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 128 is greater than input length  = 4, using nperseg = 4
 

In [51]:
columns = list()
category, video = [], []

for key in features_epochs_EEG.keys():
    user = key
    print(user)
videos = list(features_EEG[user].keys())
for epochs in features_EEG[user][videos[0]].keys():
    for bands in features_EEG[user][videos[0]][epochs].keys():
        columns.append(epochs + "_" + bands)
df_EEG = pd.DataFrame(columns=columns)

for users in features_EEG.keys():
    temp_df = pd.DataFrame(columns=columns)

    for i, videos in enumerate(features_EEG[users].keys()):
        if videos != "baseline":
            temp_list = list()
            for channel in features_EEG[users][videos].keys():
                for band in features_EEG[users][videos][channel].keys():
                    temp_list.append(
                        features_EEG[users][videos][channel][band]
                    )
            temp_df.loc[i] = list(np.array(temp_list)-np.array(EEG_list))
            category.append(videos.split("/")[1])
            video.append(videos.split("/")[2])

    df_EEG = pd.concat([df_EEG, temp_df], ignore_index=True)
df_EEG["Category"] = category
df_EEG["Video"] = video

In [52]:
df_EEG

,EEG_1_alpha,EEG_1_betha,EEG_1_gamma,EEG_1_theta,EEG_2_alpha,EEG_2_betha,EEG_2_gamma,EEG_2_theta,EEG_3_alpha,EEG_3_betha,...,EEG_31_alpha,EEG_31_betha,EEG_31_gamma,EEG_31_theta,EEG_32_alpha,EEG_32_betha,EEG_32_gamma,EEG_32_theta,Category,Video
0,0.000075,0.000058,0.000040,0.000286,0.000153,0.000147,0.000096,0.000358,0.000077,0.000142,...,0.000016,0.000015,0.000011,0.000035,0.000018,0.000014,0.000008,0.000030,Horror,1000.avi
1,0.000078,0.000057,0.000036,0.000195,0.000179,0.000153,0.000093,0.000326,0.000090,0.000222,...,0.000018,0.000014,0.000010,0.000032,0.000014,0.000014,0.000007,0.000025,Horror,1006.avi
2,0.000081,0.000054,0.000047,0.000146,0.000200,0.000138,0.000100,0.000264,0.000091,0.000102,...,0.000021,0.000013,0.000010,0.000029,0.000019,0.000012,0.000008,0.000024,Horror,1004.avi
3,0.000086,0.000077,0.000056,0.000247,0.000183,0.000190,0.000130,0.000374,0.000085,0.000184,...,0.000019,0.000019,0.000016,0.000035,0.000015,0.000022,0.000016,0.000029,Horror,1007.avi
4,0.000068,0.000059,0.000056,0.000246,0.000158,0.000135,0.000119,0.000336,0.000076,0.000108,...,0.000017,0.000013,0.000012,0.000031,0.000017,0.000013,0.000009,0.000028,Horror,1009.avi
5,0.000102,0.000080,0.000051,0.000459,0.000258,0.000190,0.000114,0.000572,0.000129,0.000121,...,0.000024,0.000017,0.000012,0.000045,0.000024,0.000019,0.000010,0.000036,Social Negative,3009.avi
6,0.000142,0.000068,0.000045,0.000377,0.000369,0.000188,0.000101,0.000576,0.000147,0.000122,...,0.000035,0.000019,0.000012,0.000051,0.000023,0.000017,0.000009,0.000034,Social Negative,3006.avi
7,0.000099,0.000086,0.000063,0.000513,0.000202,0.000207,0.000129,0.000494,0.000101,0.000174,...,0.000024,0.000020,0.000014,0.000042,0.000022,0.000021,0.000011,0.000037,Social Negative,3007.avi
8,0.000091,0.000079,0.000054,0.000373,0.000233,0.000204,0.000144,0.000523,0.000115,0.000170,...,0.000026,0.000019,0.000014,0.000072,0.000019,0.000018,0.000010,0.000054,Social Negative,3004.avi
9,0.000082,0.000056,0.000034,0.000459,0.000179,0.000134,0.000076,0.000473,0.000082,0.000073,...,0.000019,0.000014,0.000010,0.000037,0.000015,0.000014,0.000006,0.000031,Social Negative,3000.avi


In [56]:
df_baseline = pd.read_csv(r'C:\Users\Rodrigo\Desktop\PhD\loop-system\Training Models\P2\baseline.csv', sep=";", index_col=0)

In [57]:
df_baseline

,Avg HR,Min HR,Max HR,SD,AVG RR,Minimum RR,Maximum RR,SDNN,RMSSD,NN50,...,RRV_CVSD,RRV_MedianBB,RRV_MadBB,RRV_MCVBB,RRV_nn20,RRV_nn50,RRV_pNN50,RRV_pNN20,RRV_HF,RRV_SD1
0,68.003105,59.40264,82.187215,4.763109,0.886467,0.730041,1.010056,59.5714,36.5055,38,...,0.441199,478.5,84.5082,0.176611,25.0,18.0,56.25,78.125,0.078501,164.385938


In [62]:
category, videos = [], []

for key in features_signals.keys():
    user = key
video = list(features_signals[user].keys())
df = pd.DataFrame(columns=features_signals[user][video[0]].columns)
for users in features_signals.keys():
    for epoch in features_signals[users].keys():
        if epoch != "baseline":
            df = pd.concat([df, features_signals[users][epoch].sub(df_baseline)], ignore_index=True)
            category.append(epoch.split("/")[1])
            videos.append(epoch.split("/")[2])
df["Category"] = category
df["Video"] = videos

In [63]:
df

,Avg HR,Min HR,Max HR,SD,AVG RR,Minimum RR,Maximum RR,SDNN,RMSSD,NN50,...,RRV_MadBB,RRV_MCVBB,RRV_nn20,RRV_nn50,RRV_pNN50,RRV_pNN20,RRV_HF,RRV_SD1,Category,Video
0,0.351195,1.806578,8.699200,2.818434,0.000572,-0.069876,-0.029812,24.4618,0.0715,-26,...,-29.6520,-0.067553,-21.0,-15.0,3.750000,1.875000,NaN,-21.678341,Horror,1000.avi
1,0.723577,1.188288,4.747595,1.004694,-0.007839,-0.039868,-0.019809,7.6609,-1.1249,-27,...,38.5476,-0.021433,-23.0,-16.0,10.416667,-11.458333,NaN,121.114062,Horror,1006.avi
2,-3.526468,-0.011645,-10.776613,-1.875697,0.045963,0.110171,0.000198,-17.9757,-12.0555,-36,...,-63.7518,-0.123457,-20.0,-17.0,-43.750000,-15.625000,NaN,-136.972438,Horror,1004.avi
3,0.097791,0.582482,6.026201,2.113040,0.002844,-0.049872,-0.009808,22.7406,4.9328,-24,...,-20.0151,-0.023783,-20.0,-14.0,10.416667,5.208333,NaN,-78.476974,Horror,1007.avi
4,-2.968047,-0.593564,-9.915819,-1.084545,0.038998,0.100163,0.010195,-8.4642,-5.0684,-32,...,-47.4432,-0.091599,-21.0,-15.0,-13.392857,-20.982143,NaN,-114.696066,Horror,1009.avi
5,2.410832,3.739621,-0.015778,0.019361,-0.030603,0.000140,-0.059821,-4.0500,-3.8376,-29,...,148.2600,0.208767,-23.0,-16.0,10.416667,-11.458333,NaN,202.114062,Social Negative,3009.avi
6,0.799742,4.411225,-6.256792,-1.629065,-0.012613,0.060157,-0.069822,-20.0710,-10.7092,-33,...,17.0499,0.057935,-22.0,-15.0,18.750000,-3.125000,NaN,325.908022,Social Negative,3006.avi
7,8.203911,4.411924,7.343667,1.126396,-0.094317,-0.059881,-0.069832,3.0979,0.3000,-30,...,117.1254,0.125915,-22.0,-16.0,-6.250000,-3.125000,NaN,1.285927,Social Negative,3007.avi
8,2.650716,1.806751,-1.126129,0.106933,-0.033213,0.010142,-0.029814,-0.7326,4.5466,-25,...,116.3841,0.247213,-20.0,-13.0,27.083333,5.208333,NaN,-26.735475,Social Negative,3004.avi
9,1.875944,1.188344,-3.259485,-0.424469,-0.024475,0.030149,-0.019810,-5.2219,-4.6599,-29,...,25.2042,0.050537,-21.0,-14.0,10.416667,-11.458333,NaN,-69.508912,Social Negative,3000.avi


In [78]:
"""Concatenate Dataframes"""
columns = df.columns[: (len(df.columns) - 2)]
columns_EEG = df_EEG.columns[: (len(df_EEG.columns) - 2)]
full_dataframe = pd.concat([df_EEG[columns_EEG], df], axis=1)
full_columns = full_dataframe.columns[: (len(full_dataframe.columns)-2)]

In [79]:
full_columns

Index(['EEG_1_alpha', 'EEG_1_betha', 'EEG_1_gamma', 'EEG_1_theta',
       'EEG_2_alpha', 'EEG_2_betha', 'EEG_2_gamma', 'EEG_2_theta',
       'EEG_3_alpha', 'EEG_3_betha',
       ...
       'RRV_CVSD', 'RRV_MedianBB', 'RRV_MadBB', 'RRV_MCVBB', 'RRV_nn20',
       'RRV_nn50', 'RRV_pNN50', 'RRV_pNN20', 'RRV_HF', 'RRV_SD1'],
      dtype='object', length=200)

In [96]:
imp = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P2\\imp.pkl", "rb"))
scaler = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P2\\scaler.pkl", "rb"))
rfe = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P2\\rfe.pkl", "rb"))
model = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P2\\model.pkl", "rb"))

In [97]:
X = np.array(full_dataframe[full_columns])
Y = np.array(full_dataframe[["Category"]])

In [98]:
X = imp.transform(X)
X = scaler.transform(X)
X = rfe.transform(X)

prediction = model.predict(X)
prediction

array(['Social Negative', 'Social Negative', 'Social Negative',
       'Social Negative', 'Social Negative', 'Social Negative',
       'Social Negative', 'Social Negative', 'Social Negative',
       'Social Negative'], dtype=object)